In [40]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [3]:
tf.__version__


'2.15.0'

### Preprocessing

In [4]:
# import data
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [9]:
# Encoding Categorical column Gender using Label encoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [13]:
# Encoding Geography column using One hot encoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [16]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [19]:
# feature Scaling all the features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Building an ANN

In [21]:
# Initialize an ANN
# Sequential() allows to build sequential layers
ann = tf.keras.models.Sequential()
ann

In [22]:
# Adding input layer and first input layer
# dense class is used to create a fully connnected layer as a dense class object at any phase in an ann.
# add method is used to add any layer( hidden/ dropout/ conv2D) created using dense class
# When you create a first hidden layer, it automatically creates an input layer by assining
# # of neurons == # of input features. So in below, we mention # of neurons req for the first hidden layer
ann.add(tf.keras.layers.Dense(units = 9, activation = 'relu'))

In [23]:
# Add second sequnetial layer
ann.add(tf.keras.layers.Dense(units = 9, activation = 'relu'))

In [25]:
# Add output layer
# When doing multiclass classification, use softmax instead of sigmoid
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

### Training an ANN

In [27]:
# Compiling an ANN
# in this step, we mention the optimizer, loss function and metrics required to evaluate our model
# adam optimzer is used to perform Stochastic Gradient descent (in every iteration, a batch of rows gets trained
# and the weights are adjusted to reduce loss)
# When doing binary classification, we have to use binary_crossentropy loss function. If you have more than two 
# classes, then use categorical_crossentropy loss function

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [29]:
# Training an ANN
ann.fit(X_train, y_train, batch_size = 32, epochs = 300)

Epoch 1/300
250/250 [==============================] - 0s 317us/step - loss: 0.4845 - accuracy: 0.7945
Epoch 2/300
250/250 [==============================] - 0s 269us/step - loss: 0.4466 - accuracy: 0.8037
Epoch 3/300
250/250 [==============================] - 0s 270us/step - loss: 0.4316 - accuracy: 0.8083
Epoch 4/300
250/250 [==============================] - 0s 314us/step - loss: 0.4207 - accuracy: 0.8130
Epoch 5/300
250/250 [==============================] - 0s 272us/step - loss: 0.4092 - accuracy: 0.8209
Epoch 6/300
250/250 [==============================] - 0s 272us/step - loss: 0.3958 - accuracy: 0.8304
Epoch 7/300
250/250 [==============================] - 0s 282us/step - loss: 0.3790 - accuracy: 0.8414
Epoch 8/300
250/250 [==============================] - 0s 288us/step - loss: 0.3645 - accuracy: 0.8503
Epoch 9/300
250/250 [==============================] - 0s 280us/step - loss: 0.3554 - accuracy: 0.8535
Epoch 10/300
250/250 [==============================] - 0s 277us/step - l

250/250 [==============================] - 0s 283us/step - loss: 0.3262 - accuracy: 0.8651
Epoch 158/300
250/250 [==============================] - 0s 281us/step - loss: 0.3263 - accuracy: 0.8658
Epoch 159/300
250/250 [==============================] - 0s 273us/step - loss: 0.3269 - accuracy: 0.8658
Epoch 160/300
250/250 [==============================] - 0s 272us/step - loss: 0.3263 - accuracy: 0.8669
Epoch 161/300
250/250 [==============================] - 0s 271us/step - loss: 0.3264 - accuracy: 0.8661
Epoch 162/300
250/250 [==============================] - 0s 271us/step - loss: 0.3266 - accuracy: 0.8666
Epoch 163/300
250/250 [==============================] - 0s 267us/step - loss: 0.3262 - accuracy: 0.8659
Epoch 164/300
250/250 [==============================] - 0s 268us/step - loss: 0.3261 - accuracy: 0.8662
Epoch 165/300
250/250 [==============================] - 0s 268us/step - loss: 0.3262 - accuracy: 0.8664
Epoch 166/300
250/250 [==============================] - 0s 271us/ste

### Make predictions and evaluate the model



In [31]:
# Predict the result of single observation
ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))

1/1 [==============================] - 0s 11ms/step


array([[0.02632774]], dtype=float32)

Should we say goodbye to the customer

In [33]:
ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5

1/1 [==============================] - 0s 10ms/step


array([[False]])

In [35]:
# predict X_test
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 [==============================] - 0s 286us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [46]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
df = pd.DataFrame(cm)
df

,0,1
0,1507,88
1,197,208


In [44]:
# tn, fp, fn, tp
# true negatives: # of negative examples classified correctly
# false positives: # of positive examples classified incorrectly
# false negatives: # of negative exmaples classified incorrectly
# true positives: # of positive examples classified correctly
confusion_matrix(y_test, y_pred).ravel()

array([1507,   88,  197,  208])

In [42]:
# Accuracy score
accuracy_score(y_test, y_pred)

0.8575